In [1]:
from pyiron_atomistics import Project
import numpy as np

In [2]:
from pyiron_workflow import Workflow
from pyironflow.pyironflow import PyironFlow

In [3]:
@Workflow.wrap.as_function_node
def CreateProject(pr_name: str, element: str):
    pr = Project(pr_name)
    pr_el = {'project': pr, 'element': element}
    return pr_el

In [15]:
@Workflow.wrap.as_function_node
def CreateSample(pr_el, cubic: bool = False):
    pr = pr_el['project']
    element = pr_el['element']
    structure = pr.create.structure.bulk(element, cubic=cubic)
    return structure

In [16]:
@Workflow.wrap.as_function_node("View")
def VisualizeStructure(structure):
    return structure.plot3D

In [17]:
@Workflow.wrap.as_function_node
def LammpsCalcMinimize(pr_el, structure, job_name:str, pot_list_index: int, f_tol: float, min_style: str, del_ex_job: bool = False, print_concept_dicts: bool = False):
    pr = pr_el['project']
    job = pr.create.job.Lammps(job_name, delete_existing_job=del_ex_job)
    job.structure = structure
    job.potential = job.list_potentials()[pot_list_index]
    job.calc_minimize(f_tol=f_tol, style=min_style)
    job.run()
    
    import os
    os.system('conda env export | grep -v "^prefix: " > ' + job.project.name + '/' + job.name + '_environment.yml')
    
    from pyiron_base.storage.hdfio import FileHDFio
    hdf = FileHDFio(job.project.name + '/' + job.name + '_input_structure.h5')
    job.structure.to_hdf(hdf)

    import json
    cdict = process_lammps_job(job)
    file_name = job.project.name + '/' + job.name + '_concept_dict.json'
    with open(file_name, 'w') as f:
        json.dump(cdict, f, indent=2)
        
    struct_cdict = process_structure_crystal(job)
    struct_file_name = job.project.name + '/' + job.name + '_input_structure_concept_dict.json'
    with open(struct_file_name, 'w') as f:
        json.dump(struct_cdict, f, indent=2)
        
    if print_concept_dicts == True: 
        print(json.dumps(struct_cdict, indent=2))
        print(json.dumps(cdict, indent=2))
        
    return job

In [18]:
@Workflow.wrap.as_function_node("view")
def ViewStructure(structure):
    return structure.plot3d()

In [19]:
@Workflow.wrap.as_function_node("object")
def LammpsJobOpenBIS(username:str, space:str, project:str, collection:str, job, separate_input_sample: bool = True):
    import json
    cdict_file = job.path + '_concept_dict.json'
    struct_cdict_file = job.path + '_input_structure_concept_dict.json'
    try:
        with open(cdict_file) as json_file:
            concept_dict = json.load(json_file)
        with open(struct_cdict_file) as json_file:
            struct_concept_dict = json.load(json_file)
    except:
        concept_dict = process_lammps_job(job)
        file_name = job.project.name + '/' + job.name + '_concept_dict.json'
        with open(file_name, 'w') as f:
            json.dump(concept_dict, f, indent=2)
            
        struct_concept_dict = process_structure_crystal(job)
        struct_file_name = job.project.name + '/' + job.name + '_input_structure_concept_dict.json'
        with open(struct_file_name, 'w') as f:
            json.dump(struct_concept_dict, f, indent=2)

    from IPython.display import display
    if separate_input_sample == True:
        s = GenericCrystalObject(username, space, project, collection, "input", struct_concept_dict)
        j = GenericLammpsJobObject(username, space, project, collection, concept_dict, struct_concept_dict)
        display(s)
        display(j)
    else:
        j = GenericLammpsJobObject(username, space, project, collection, concept_dict)
        display(j)
            
    return "Done"

In [40]:
wf = Workflow("test_wf")

wf.Project_KG_Element = CreateProject("test_gui", "Fe")
wf.Structure = CreateSample(wf.Project_KG_Element, True)
wf.ViewStructure = ViewStructure(wf.Structure)
#wf.SampleKG = SampleKG(wf.Project_KG_Element)
wf.LammpsCalcMinimize = LammpsCalcMinimize(wf.Project_KG_Element, wf.Structure, "test_lammps_gui", 68, 1e-8, "fire", True, False)
#wf.JobKG = JobKG(wf.Project_KG_Element, wf.LammpsCalcMinimize)
wf.LammpsJobOpenBIS = LammpsJobOpenBIS("tlakshmi", "TLAKSHMI", "TEST", "TEST_EXP_1", wf.LammpsCalcMinimize)

In [44]:
#wf.LammpsJobOpenBIS.pull()

In [21]:
for v in wf.children.values():
    for i in v.inputs:
        for c in i.connections:
            print(v.label, v.inputs, c.owner.label)

Structure Inputs ['pr_el', 'cubic'] Project_KG_Element
ViewStructure Inputs ['structure'] Structure
LammpsCalcMinimize Inputs ['pr_el', 'structure', 'job_name', 'pot_list_index', 'f_tol', 'min_style', 'del_ex_job', 'print_concept_dicts'] Project_KG_Element
LammpsCalcMinimize Inputs ['pr_el', 'structure', 'job_name', 'pot_list_index', 'f_tol', 'min_style', 'del_ex_job', 'print_concept_dicts'] Structure
LammpsJobOpenBIS Inputs ['username', 'space', 'project', 'collection', 'job', 'separate_input_sample'] LammpsCalcMinimize


In [22]:
pf = PyironFlow([wf])
pf.gui

In [23]:
pf.out_widget

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [41]:
from pybis import Openbis
o = Openbis('https://test3.datastore.bam.de/')          # https:// is assumed

import getpass
password = getpass.getpass()

o.login('tlakshmi', password, save_token=True)   # save the session token in ~/.pybis/example.com.token

 ········


'tlakshmi-241024102500443x45A4AED1B3EC6101BC0AD7FE8BC7A065'

In [244]:
#o.logout()

In [42]:
print(f"Session is active: {o.is_session_active()} and token is {o.token}")

Session is active: True and token is tlakshmi-241024102500443x45A4AED1B3EC6101BC0AD7FE8BC7A065


In [246]:
#obis_obj = o.get_sample("/TLAKSHMI/TEST/TEST_EXP_1/PYI_JOB.LMP81")
#obis_obj.p["description"]

In [247]:
#obis_obj.set_properties({'description': '<figure class="image image_resized image-style-align-left" style="width:11.48%;"><img src="/openbis/openbis/file-service/eln-lims/30/77/c8/3077c8f8-0e55-41e5-9021-7ed43863e5a4/cc44149e-b2d4-4850-9f88-7a4cd54c1a66.png"></figure><p>Lammps simulation using pyiron for energy minimization/structural optimization.<br>&nbsp;</p><p><span style="color:hsl(240,75%,60%);"><strong>Scroll down below other properties to view conceptual dictionary with ontological ids of selected properties and values.</strong></span></p><p>The conceptual dictionary is in JSON-LD format. Learn more about it <a href="https://www.w3.org/ns/json-ld/">here</a><br>&nbsp;</p>'})
#obis_obj.save()

In [248]:
wf_from_gui = pf.get_workflow()

In [249]:
wf_from_gui.children.items()

dict_items([('Project_KG_Element', <__main__.CreateProject object at 0x7f5a73fcbd10>), ('Structure', <__main__.CreateSample object at 0x7f5a7409d110>), ('ViewStructure', <__main__.ViewStructure object at 0x7f5a73fcb5d0>), ('LammpsCalcMinimize', <__main__.LammpsCalcMinimize object at 0x7f5a73fb3390>), ('JobKG', <__main__.JobKG object at 0x7f5a73f88090>), ('LammpsJobOpenBIS', <__main__.LammpsJobOpenBIS object at 0x7f5a73fb2a50>)])

In [250]:
obis_job = wf_from_gui.LammpsCalcMinimize.pull()

/home/tlakshmi/anaconda3/envs/pyiron_unify_1/lib/python3.11/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['international']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(
/home/tlakshmi/anaconda3/envs/pyiron_unify_1/lib/python3.11/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['number']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(
/home/tlakshmi/anaconda3/envs/pyiron_unify_1/lib/python3.11/site-packages/pyiron_atomistics/lammps/interactive.py:302: UserWarning: f_tol is deprecated as of vers. 0.3.0. It is not guaranteed to be in service in vers. 0.4.0. Use ionic_force_tolerance instead.
  warnings.warn(


The job test_lammps_gui was saved and received the ID: 49


In [33]:
def GenericCrystalObject(user, space, project, collection, name_prefix, concept_dict):
    o = Openbis('https://test3.datastore.bam.de/')
    cdict = flatten_crystal_cdict(concept_dict)

    objects = o.get_objects(
        space      = space,
        type       ='MAT_SIM_STRUCTURE.CRYSTAL',
        start_with = 0,
    )
    exists = False
    for object_ in objects:
        if object_.p.get('$name') == name_prefix + '_structure_' + cdict['job_name']:
            exists = True
            found_object = object_

    if exists == True:
        print("=========================\n")
        print("Structure already exists!\n")
        print("=========================\n")
        print("Found structure properties:\n")
        return found_object.p

    else:

        description =   '<figure class="image image_resized image-style-align-left" style="width:9%;">' + \
                        '<img src="/openbis/openbis/file-service/eln-lims/30/77/c8/3077c8f8-0e55-41e5-9021-7ed43863e5a4/cc44149e-b2d4-4850-9f88-7a4cd54c1a66.png">' + \
                        '</figure><p><br>Crystal structure generated using pyiron.<br>&nbsp;</p><p><span style="color:hsl(240,75%,60%);">' + \
                        '<strong>Scroll down below other properties to view conceptual dictionary with ontological ids of selected properties and values.</strong></span>' + \
                        '</p><p>The conceptual dictionary is in JSON-LD format. Learn more about it <a href="https://www.w3.org/ns/json-ld/">here</a><br>&nbsp;</p>'
        
        species = {}
        for i in concept_dict['atoms']:
            if i['label'] != 'total_number_atoms':
                species[i['label']] = i['value']

        json_file = cdict['project_name'] + '/' + cdict['job_name'] + "_" + name_prefix + '_structure_concept_dict.json'
        with open(json_file, 'r') as file:
            json_string = file.read()
        json_string = format_json_string(json_string)

        props_dict = {
            '$name': name_prefix + '_structure_' + cdict['job_name'],
            'description': description,
            'workflow_manager': cdict['workflow_manager'],
            'chem_species_by_n_atoms': str(species),
            'n_atoms_total': cdict['total_number_atoms'],
            'sim_cell_lengths_in_a': cdict['simulation_cell_lengths'],
            'sim_cell_vectors': cdict['simulation_cell_vectors'],
            'sim_cell_angles_in_deg': cdict['simulation_cell_angles'],
            'sim_cell_volume_in_a3': cdict['simulation_cell_volume'],
            'conceptual_dictionary': json_string,
        }

        object_ = o.new_object(
            type       = 'MAT_SIM_STRUCTURE.CRYSTAL',
            space      = space,
            experiment = '/' + space + '/' + project + '/' + collection,
            props      = props_dict
        )
        object_.save()

        dataset_props_dict = {
            '$name': name_prefix + '_structure_' + cdict['job_name'] + '.h5',
            'multi_mat_scale': 'Electronic/Atomistic',
            'sw_compatibility': 'ASE',
            'file_format': 'HDF5',
        }
        
        path_to_h5 = cdict['project_name'] + '/' + str(cdict['job_name']) + '_input_structure.h5'
        
        ds_hdf = o.new_dataset(
            type       = 'MAT_SIM_STRUCTURE',
            collection = '/' + space + '/' + project + '/' + collection,
            object     = object_,
            files      = [path_to_h5],
            props      = dataset_props_dict
        )

        ds_hdf.save()

    
    return object_.p

In [34]:
def GenericLammpsJobObject(user, space, project, collection, concept_dict, struct_concept_dict = None):
    o = Openbis('https://test3.datastore.bam.de/')
    cdict = flatten_lammps_cdict(concept_dict)

    objects = o.get_objects(
        space      = space,
        type       ='PYIRON_JOB.LAMMPS',
        start_with = 0,
    )
    exists = False
    for object_ in objects:
        if object_.p.get('$name') == cdict['job_name']:
            exists = True
            found_object = object_

    if exists == True:
        print("===================\n")
        print("Job already exists!\n")
        print("===================\n")
        print("Found job properties:\n")
        return found_object.p
    
    else:

        if cdict['job_status'] == 'finished':
            job_status = True
        else:
            job_status = False

        from datetime import datetime
        delta = datetime.strptime(cdict['job_stoptime'], "%Y-%m-%d %H:%M:%S") - datetime.strptime(cdict['job_starttime'], "%Y-%m-%d %H:%M:%S")

        atom_calc_type = None
        if 'molecular_statics' in concept_dict.keys() and 'minimization_algorithm' in cdict.keys():
            atom_calc_type = ('ATOM_CALC_STRUC_OPT').lower()
            description =   '<figure class="image image_resized image-style-align-left" style="width:9%;">' + \
                            '<img src="/openbis/openbis/file-service/eln-lims/30/77/c8/3077c8f8-0e55-41e5-9021-7ed43863e5a4/cc44149e-b2d4-4850-9f88-7a4cd54c1a66.png">' + \
                            '</figure><p><br>Lammps simulation using pyiron for energy minimization/structural optimization.<br>&nbsp;</p><p><span style="color:hsl(240,75%,60%);">' + \
                            '<strong>Scroll down below other properties to view conceptual dictionary with ontological ids of selected properties and values.</strong></span>' + \
                            '</p><p>The conceptual dictionary is in JSON-LD format. Learn more about it <a href="https://www.w3.org/ns/json-ld/">here</a><br>&nbsp;</p>'

        atom_ionic_min_algo = None
        if cdict['minimization_algorithm'] == 'fire':
            atom_ionic_min_algo = ('MIN_ALGO_FIRE').lower()

        json_file = cdict['project_name'] + '/' + cdict['job_name'] + '_concept_dict.json'
        with open(json_file, 'r') as file:
            json_string = file.read()
        json_string = format_json_string(json_string)

        props_dict = {
            '$name': cdict['job_name'],
            'description': description,
            'workflow_manager': cdict['workflow_manager'],
            'bam_username': user,
            'sim_job_finished': job_status,
            'start_date': cdict['job_stoptime'],
            'sim_walltime_in_hours': delta.total_seconds()/3600,
            'sim_coretime_in_hours': cdict['sim_coretime_hours'],
            'ncores': cdict['number_cores'],
            'atomistic_calc_type': atom_calc_type,
            'periodic_boundary_x': cdict['periodicity_in_x'],
            'periodic_boundary_y': cdict['periodicity_in_y'],
            'periodic_boundary_z': cdict['periodicity_in_z'],
            'atom_cell_vol_relax': True if 'http://purls.helmholtz-metadaten.de/asmo/CellVolumeRelaxation' in cdict['dof'] else False,
            'atom_cell_shp_relax': True if 'CellShapeRelaxation' in cdict['dof'] else False,
            'atom_pos_relax': True if 'http://purls.helmholtz-metadaten.de/asmo/AtomicPositionRelaxation' in cdict['dof'] else False,
            'atom_ionic_min_algo': atom_ionic_min_algo,
            'max_iters': cdict['maximum iterations'],
            'atom_e_tol_ion_in_ev': cdict['ionic energy tolerance'],
            'atom_f_tol_in_ev_a': cdict['force tolerance'],
            'atom_ionic_steps': cdict['NumberIonicSteps'],
            'atom_fin_tot_eng_in_ev': cdict['FinalTotalEnergy'],
            'atom_fin_vol_in_a3': cdict['FinalTotalVolume'],
            'atom_fin_pot_eng_in_ev': cdict['FinalPotentialEnergy'],
            'atom_force_max_in_ev_a': cdict['FinalMaximumForce'],
            'conceptual_dictionary': json_string,
        }
    
        object_ = o.new_object(
            type       = 'PYIRON_JOB.LAMMPS',
            space      = space,
            experiment = '/' + space + '/' + project + '/' + collection,
            props      = props_dict
        )
        object_.save()

        #hdf_ver = job.to_dict()['HDF_VERSION']
        path_to_h5 = cdict['project_name'] + '/' + str(cdict['job_name']) + '.h5'
        path_to_json = cdict['project_name'] + '/' + str(cdict['job_name']) + '_concept_dict.json'
        path_to_yml = cdict['project_name'] + '/' + str(cdict['job_name']) + '_environment.yml'
        
        dataset_props_dict = {
            '$name': cdict['job_name'] + '.h5',
            'production_date': datetime.strptime(cdict['job_stoptime'], "%Y-%m-%d %H:%M:%S").date().strftime("%Y-%m-%d"),
            'file_format': 'HDF5',
            #'hdf5_version': hdf_ver,
            'reference': 'https://github.com/pyiron/pyiron_atomistics/blob/main/pyiron_atomistics/lammps/base.py',
        }
        
        ds_hdf = o.new_dataset(
            type       = 'PYIRON_JOB',
            collection = '/' + space + '/' + project + '/' + collection,
            object     = object_,
            files      = [path_to_h5],
            props      = dataset_props_dict
        )

        ds_hdf.save()

        ds_json = o.new_dataset(
            type       = 'ATTACHMENT',
            collection = '/' + space + '/' + project + '/' + collection,
            object     = object_,
            files      = [path_to_json],
            props      = {'$name':cdict['job_name']+'_concept_dict.json'}
        )

        ds_json.save()

        ds_yml = o.new_dataset(
            type       = 'COMP_ENV',
            collection = '/' + space + '/' + project + '/' + collection,
            object     = object_,
            files      = [path_to_yml],
            props      = {'$name':cdict['job_name']+'_environment.yml', 'env_tool': 'conda'}
        )

        ds_yml.save()

        if struct_concept_dict != None:
            struct_object_name = 'input_structure_' + flatten_crystal_cdict(struct_concept_dict)['job_name']
            objects_1 = o.get_objects(
                space      = space,
                type       ='MAT_SIM_STRUCTURE.CRYSTAL',
                start_with = 0,
            )
            exists = False
            for object_1 in objects_1:
                if object_1.p.get('$name') == struct_object_name:
                    exists = True
                    found_object_1 = object_1

            if exists == True:
                object_.set_parents(found_object_1.identifier)
                object_.save()
            else:
                print("==============================\n")
                print("Create structure object first!\n")
                print("==============================\n")
        
        return object_.p

In [32]:
"""
LAMMPS specific functions for parsing

Use this a reference for specific implementations
"""
import os
import numpy as np
import ast


def process_structure_crystal(job, initial_structure = None, rotation_indices = None):
    sample_dict = {}
    add_structure_contexts(sample_dict)
    get_chemical_species(job, sample_dict)
    get_simulation_cell(job, sample_dict)
    add_structure_software(job, sample_dict)
    get_structure_folder(job, sample_dict)
    return sample_dict

def add_structure_contexts(sample_dict):
    sample_dict['@context'] = {}
    sample_dict['@context']['path'] = 'http://purls.helmholtz-metadaten.de/cmso/hasPath'
    sample_dict['@context']['unit_cell'] = 'http://purls.helmholtz-metadaten.de/cmso/UnitCell'
    sample_dict['@context']['atoms'] = 'http://purls.helmholtz-metadaten.de/cmso/Atom'
    sample_dict['@context']['molecules'] = 'http://purls.helmholtz-metadaten.de/cmso/Molecule'
    sample_dict['@context']['bravais_lattice'] = 'http://purls.helmholtz-metadaten.de/cmso/hasBravaisLattice'
    sample_dict['@context']['chemical_species'] = 'http://purls.helmholtz-metadaten.de/cmso/ChemicalSpecies'
    sample_dict['@context']['simulation_cell'] = 'http://www.w3.org/2000/01/rdf-schema#label'
    sample_dict['@context']['label'] = 'http://www.w3.org/2000/01/rdf-schema#label'
    sample_dict['@context']['unit'] = 'http://purls.helmholtz-metadaten.de/cmso/hasUnit'
    sample_dict['@context']['value'] = 'http://purls.helmholtz-metadaten.de/asmo/hasValue'
    sample_dict['@context']['vector'] = 'http://purls.helmholtz-metadaten.de/cmso/Vector'
    #method_dict['@context']['workflow_manager'] = ''
    #method_dict['@context']['software'] = ''


def get_structure_folder(job, sample_dict):
    sample_dict['path'] = os.path.join(job.project.path, f'{job.name}_hdf5')

def identify_unit_cell(job, sample_dict):
    #Stuff lattice quantities and orientation
    return

def identify_crystal_structure(job, sample_dict):
    #Stuff here for space group and bravais lattice
    return

def get_chemical_species(job, sample_dict):
    structure = job.structure
    natoms = structure.get_number_of_atoms()
    species_dict = dict(structure.get_number_species_atoms())
    atoms_list = []
    for k in species_dict.keys():
        element = {}
        element["value"] = species_dict[k]
        element["label"] = k
        atoms_list.append(element)

    atoms_list.append({'value': natoms, 'label': 'total_number_atoms'})
        
    sample_dict['atoms'] = atoms_list

def get_simulation_cell(job, sample_dict):
    structure = job.structure
    cell_lengths = str([structure.cell.cellpar()[0],structure.cell.cellpar()[1],structure.cell.cellpar()[2]])
    cell_vectors = str([structure.cell[0],structure.cell[1],structure.cell[2]])
    cell_angles = str([structure.cell.cellpar()[3],structure.cell.cellpar()[4],structure.cell.cellpar()[5]])
    cell_volume = structure.get_volume()
    
    simulation_cell_details = []

    cell_lengths_dict = {}
    cell_lengths_dict["value"] = cell_lengths
    cell_lengths_dict["unit"] = "ANGSTROM"
    cell_lengths_dict["label"] = 'simulation_cell_lengths'
    cell_lengths_dict["@id"] = "http://purls.helmholtz-metadaten.de/cmso/hasLength"
    simulation_cell_details.append(cell_lengths_dict)
    
    cell_vector_dict = {}
    cell_vector_dict["value"] = cell_vectors
    cell_vector_dict["unit"] = "ANGSTROM"
    cell_vector_dict["label"] = 'simulation_cell_vectors'
    cell_vector_dict["@id"] = "http://purls.helmholtz-metadaten.de/cmso/hasVector"
    simulation_cell_details.append(cell_vector_dict)

    cell_angles_dict = {}
    cell_angles_dict["value"] = cell_angles
    cell_angles_dict["unit"] = "DEGREES"
    cell_angles_dict["label"] = 'simulation_cell_angles'
    cell_angles_dict["@id"] = "http://purls.helmholtz-metadaten.de/cmso/hasAngle"
    simulation_cell_details.append(cell_angles_dict)

    cell_volume_dict = {}
    cell_volume_dict["value"] = np.round(cell_volume, decimals=4)
    cell_volume_dict["unit"] = "ANGSTROM3"
    cell_volume_dict["label"] = 'simulation_cell_volume'
    cell_volume_dict["@id"] = "http://purls.helmholtz-metadaten.de/cmso/hasVolume"
    simulation_cell_details.append(cell_volume_dict)

    sample_dict['simulation_cell'] = simulation_cell_details

def add_structure_software(job, sample_dict):
    sample_dict["workflow_manager"] = {}
    sample_dict["workflow_manager"]["@id"] = "http://demo.fiz-karlsruhe.de/matwerk/E457491"
    import subprocess
    try:
        output1 = subprocess.check_output(['grep', 'pyiron_atomistics', job.project.name + '/' + job.name + '_environment.yml'])
        s1 = str((output1.decode('utf-8')))
    except:
        s1 = ''
    try:     
        output2 = subprocess.check_output(['grep', 'pyiron_workflow', job.project.name + '/' + job.name + '_environment.yml'])
        s2 = str((output2.decode('utf-8')))
    except:
        s2 = ''
    hdf_ver = job.to_dict()['HDF_VERSION']
    st = 'p' + s1.split('=')[0].split('p')[1] + "=" + s1.split('=')[1] + ', ' + 'p' + s2.split('=')[0].split('p')[1] + "=" + s2.split('=')[1] #+ ', pyiron_HDF_version=' + hdf_ver
    sample_dict["workflow_manager"]["label"] = st
    
    pyiron_job_details = []
    pyiron_job_details.append(
        {
            "label": "job_name",
            "value": job.name,
        }
    )
    pyiron_job_details.append(
        {
            "label": "project_name",
            "value": job.project.name,
        }
    )
    sample_dict["job_details"] = pyiron_job_details


In [31]:
"""
LAMMPS specific functions for parsing

Use this a reference for specific implementations
"""
import os
import numpy as np
import ast


def process_lammps_job(job):
    method_dict = {}
    add_job_contexts(method_dict)
    #get_structures(job, method_dict)
    identify_method(job, method_dict)
    extract_calculated_quantities(job, method_dict)
    add_job_software(job, method_dict)
    #add_pyiron_details(job, method_dict)
    #add_sim_details(job, method_dict)
    get_simulation_folder(job, method_dict)
    return method_dict

def add_job_contexts(method_dict):
    method_dict['@context'] = {}
    method_dict['@context']['sample'] = 'http://purls.helmholtz-metadaten.de/cmso/AtomicScaleSample'
    method_dict['@context']['path'] = 'http://purls.helmholtz-metadaten.de/cmso/hasPath'
    method_dict['@context']['dof'] = 'http://purls.helmholtz-metadaten.de/asmo/hasRelaxationDOF'
    method_dict['@context']['inputs'] = 'http://purls.helmholtz-metadaten.de/asmo/hasInputParameter'
    method_dict['@context']['label'] = 'http://www.w3.org/2000/01/rdf-schema#label'
    method_dict['@context']['unit'] = 'http://purls.helmholtz-metadaten.de/asmo/hasUnit'
    method_dict['@context']['value'] = 'http://purls.helmholtz-metadaten.de/asmo/hasValue'
    method_dict['@context']['outputs'] = 'http://purls.helmholtz-metadaten.de/cmso/hasCalculatedProperty'
    #method_dict['@context']['workflow_manager'] = ''
    #method_dict['@context']['software'] = ''
    method_dict['@context']['molecular_dynamics'] = 'http://purls.helmholtz-metadaten.de/asmo/MolecularDynamics'
    method_dict['@context']['molecular_statics'] = 'http://purls.helmholtz-metadaten.de/asmo/MolecularStatics'
    method_dict['@context']['ensemble'] = 'http://purls.helmholtz-metadaten.de/asmo/hasStatisticalEnsemble'


def get_simulation_folder(job, method_dict):
    method_dict['path'] = os.path.join(job.project.path, f'{job.name}_hdf5')

def get_structures(job, method_dict):
    initial_pyiron_structure = job.structure
    final_pyiron_structure = job.get_structure(frame=-1)
    
    method_dict['sample'] =  {'initial':initial_pyiron_structure, 
                            'final': final_pyiron_structure}

def identify_method(job, method_dict):
    job_dict = job.input.to_dict()
    input_dict = {
        job_dict["control_inp/data_dict"]["Parameter"][x]: job_dict[
            "control_inp/data_dict"
        ]["Value"][x]
        for x in range(len(job_dict["control_inp/data_dict"]["Parameter"]))
    }
    dof = []
    temp = None
    press = None
    md_method = None
    ensemble = None

    if "min_style" in input_dict.keys():
        dof.append("http://purls.helmholtz-metadaten.de/asmo/AtomicPositionRelaxation")
        if job.input.control['fix___ensemble'] != 'all nve': dof.append("http://purls.helmholtz-metadaten.de/asmo/CellVolumeRelaxation")
        md_method = "molecular_statics"
        e_tol = float(input_dict['minimize'].split()[0])
        f_tol = float(input_dict['minimize'].split()[1])
        maxiter = int(input_dict['minimize'].split()[2])

    elif "nve" in input_dict["fix___ensemble"]:
        if int(input_dict["run"]) == 0:
            method = "static"
            md_method = "molecular_statics"
            ensemble = "http://purls.helmholtz-metadaten.de/asmo/MicrocanonicalEnsemble"

        elif int(input_dict["run"]) > 0:
            method = "md_nve"
            dof.append("http://purls.helmholtz-metadaten.de/asmo/AtomicPositionRelaxation")
            md_method = "molecular_dynamics"
            ensemble = "http://purls.helmholtz-metadaten.de/asmo/MicrocanonicalEnsemble"

    elif "nvt" in input_dict["fix___ensemble"]:
        method = "md_nvt"
        raw = input_dict["fix___ensemble"].split()
        temp = float(raw[3])
        dof.append("http://purls.helmholtz-metadaten.de/asmo/AtomicPositionRelaxation")
        md_method = "molecular_dynamics"
        ensemble = "http://purls.helmholtz-metadaten.de/asmo/CanonicalEnsemble"

    elif "npt" in input_dict["fix___ensemble"]:
        dof.append("http://purls.helmholtz-metadaten.de/asmo/AtomicPositionRelaxation")
        dof.append("http://purls.helmholtz-metadaten.de/asmo/CellVolumeRelaxation")
        if "aniso" in input_dict["fix___ensemble"]:
            method = "md_npt_aniso"
            dof.append("http://purls.helmholtz-metadaten.de/asmo/CellShapeRelaxation")
        else:
            method = "md_npt_iso"
        md_method = "molecular_dynamics"
        raw = input_dict["fix___ensemble"].split()
        temp = float(raw[3])
        press = float(raw[7])
        ensemble = "http://purls.helmholtz-metadaten.de/asmo/IsothermalIsobaricEnsemble"

    method_dict[md_method] = {}

    if md_method == "molecular_statics":
        method_dict[md_method]['minimization_algorithm'] = input_dict['min_style']

    input_dict = {
        job_dict["control_inp/data_dict"]["Parameter"][x]: job_dict[
            "control_inp/data_dict"
        ]["Value"][x]
        for x in range(len(job_dict["control_inp/data_dict"]["Parameter"]))
    }
    pb = []
    pb.append(input_dict['boundary'])
    if (pb[0][0] == "p"):
        method_dict[md_method]['periodicity_in_x'] = True
    else:
        method_dict[md_method]['periodicity_in_x'] = False
    if (pb[0][2] == "p"):
        method_dict[md_method]['periodicity_in_y'] = True
    else:
        method_dict[md_method]['periodicity_in_y'] = False
    if (pb[0][4] == "p"):
        method_dict[md_method]['periodicity_in_z'] = True
    else:
        method_dict[md_method]['periodicity_in_z'] = False

    method_dict[md_method]['inputs'] = []

    temperature = {}
    temperature["value"] = temp
    temperature["unit"] = "K"
    temperature["label"] = "temperature"

    method_dict[md_method]['inputs'].append(temperature)

    pressure = {}
    pressure["value"] = press
    pressure["unit"] = "GigaPA"
    pressure["label"] = "pressure"

    method_dict[md_method]['inputs'].append(pressure)

    energy_tol = {}
    energy_tol["value"] = e_tol
    energy_tol["unit"] = "EV"
    energy_tol["label"] = "ionic energy tolerance"

    method_dict[md_method]['inputs'].append(energy_tol)

    force_tol = {}
    force_tol["value"] = f_tol
    force_tol["unit"] = "EV/ANGSTROM"
    force_tol["label"] = "force tolerance"

    method_dict[md_method]['inputs'].append(force_tol)

    maximum_iterations = {}
    maximum_iterations["value"] = maxiter
    maximum_iterations["label"] = "maximum iterations"

    method_dict[md_method]['inputs'].append(maximum_iterations)

    method_dict[md_method]["ensemble"] = ensemble
    
    method_dict["dof"] = dof


    # now process potential
    inpdict = job.input.to_dict()
    ps = inpdict["potential_inp/data_dict"]["Value"][0]
    name = inpdict["potential_inp/potential/Name"]
    potstr = job.input.to_dict()["potential_inp/potential/Citations"]
    potdict = ast.literal_eval(potstr[1:-1])
    url = None
    if "url" in potdict[list(potdict.keys())[0]].keys():
        url = potdict[list(potdict.keys())[0]]["url"]

    if 'meam' in ps:
        method_dict['@context']['potential'] = "http://purls.helmholtz-metadaten.de/asmo/ModifiedEmbeddedAtomModel"
    elif 'eam' in ps:
        method_dict['@context']['potential'] = "http://purls.helmholtz-metadaten.de/asmo/EmbeddedAtomModel"
    elif 'lj' in ps:
        method_dict['@context']['potential'] = "http://purls.helmholtz-metadaten.de/asmo/LennardJonesPotential"
    elif 'ace' in ps:
        method_dict['@context']['potential'] = "http://purls.helmholtz-metadaten.de/asmo/MachineLearningPotential"
    else:
        method_dict['@context']['potential'] = "http://purls.helmholtz-metadaten.de/asmo/InteratomicPotential"


    method_dict[md_method]["potential"] = {}
    method_dict[md_method]["potential"]["label"] = name
    if url is not None:
        method_dict[md_method]["potential"]["@id"] = url

def add_job_software(job, method_dict):
    method_dict["workflow_manager"] = {}
    method_dict["workflow_manager"]["@id"] = "http://demo.fiz-karlsruhe.de/matwerk/E457491"
    import subprocess
    try:
        output1 = subprocess.check_output(['grep', 'pyiron_atomistics', job.project.name + '/' + job.name + '_environment.yml'])
        s1 = str((output1.decode('utf-8')))
    except:
        s1 = ''
    try:     
        output2 = subprocess.check_output(['grep', 'pyiron_workflow', job.project.name + '/' + job.name + '_environment.yml'])
        s2 = str((output2.decode('utf-8')))
    except:
        s2 = ''
    hdf_ver = job.to_dict()['HDF_VERSION']
    st = 'p' + s1.split('=')[0].split('p')[1] + "=" + s1.split('=')[1] + ', ' + 'p' + s2.split('=')[0].split('p')[1] + "=" + s2.split('=')[1] #+ ', pyiron_HDF_version=' + hdf_ver
    method_dict["workflow_manager"]["label"] = st

    pyiron_job_details = []
    pyiron_job_details.append(
        {
            "label": "job_name",
            "value": job.name,
        }
    )
    pyiron_job_details.append(
        {
            "label": "project_name",
            "value": job.project.name,
        }
    )
    pyiron_job_details.append(
        {
            "label": "job_type",
            "value": job.database_entry.hamilton,
        }
    )
    pyiron_job_details.append(
        {
            "label": "job_status",
            "value": str(job.status),
        }
    )
    pyiron_job_details.append(
        {
            "label": "job_starttime",
            "value": str(job.database_entry.timestart.strftime("%Y-%m-%d %H:%M:%S")),
        }
    )
    pyiron_job_details.append(
        {
            "label": "job_stoptime",
            "value": str(job.database_entry.timestop.strftime("%Y-%m-%d %H:%M:%S")),
        }
    )
    pyiron_job_details.append(
        {
            "label": "sim_coretime_hours",
            "value": job.database_entry.totalcputime,
        }
    )
    pyiron_job_details.append(
        {
            "label": "number_cores",
            "value": job.to_dict()['server']['cores'],
        }
    )
    method_dict["job_details"] = pyiron_job_details
    # and finally code details

    software = {
        "@id": "http://demo.fiz-karlsruhe.de/matwerk/E447986",
        "label": "LAMMPS " + job.to_dict()['executable']['version'],
    }
    method_dict["software"] = [software]

def extract_calculated_quantities(job, method_dict):
    """
    Extracts calculated quantities from a job.

    Parameters
    ----------
    job : pyiron.Job
        The job object containing the calculated quantities.

    Returns
    -------
    list
        A list of dictionaries, each containing the label, value, unit, and associate_to_sample of a calculated quantity.

    """
    aen = np.mean(job.output.energy_tot)
    fen = job.output.energy_tot[-1]
    fpe = job.output.energy_tot[-1]
    avol = np.mean(job.output.volume)
    fvol = job.output.volume[-1]
    fmax = job.output.force_max[-1]
    nionic = len(job.output.steps)-1
    outputs = []
    outputs.append(
        {
            "label": "AverageTotalEnergy",
            "value": np.round(aen, decimals=4),
            "unit": "EV",
        }
    )
    outputs.append(
        {
            "label": "FinalTotalEnergy",
            "value": np.round(fen, decimals=4),
            "unit": "EV",
        }
    )
    outputs.append(
        {
            "label": "FinalPotentialEnergy",
            "value": np.round(fpe, decimals=4),
            "unit": "EV",
        }
    )
    outputs.append(
        {
            "label": "AverageTotalVolume",
            "value": np.round(avol, decimals=4),
            "unit": "ANGSTROM3",
        }
    )
    outputs.append(
        {
            "label": "FinalTotalVolume",
            "value": np.round(fvol, decimals=4),
            "unit": "ANGSTROM3",
        }
    )
    outputs.append(
        {
            "label": "FinalMaximumForce",
            "value": np.round(fmax, decimals=16),
            "unit": "EV/ANGSTROM",
        }
    )
    outputs.append(
        {
            "label": "NumberIonicSteps",
            "value": nionic,
        }
    )
    method_dict['outputs'] =  outputs

In [27]:
def flatten_lammps_cdict(cdict):
        flat = {}
        for k, v in cdict.items():
            if k != '@context':
                if isinstance(v, dict):
                    if 'label' in v.keys():
                        flat[k] = v['label']
                    else:
                        flat = flat | flatten_lammps_cdict(v)
                elif k == 'inputs' or k == 'outputs' or k == 'job_details':
                    for i in v:
                        flat[i['label']] = i['value']
                elif k == 'software':
                    flat[k] = v[0]['label']
                else:
                    flat[k] = v
        return flat

In [28]:
def flatten_crystal_cdict(cdict):
        flat = {}
        for k, v in cdict.items():
            if k != '@context':
                if isinstance(v, dict):
                    if 'label' in v.keys():
                        flat[k] = v['label']
                    else:
                        flat = flat | flatten_crystal_cdict(v)
                elif k == 'atoms' or k == 'simulation_cell' or k == 'job_details':
                    for i in v:
                        flat[i['label']] = i['value']
                elif k == 'software':
                    flat[k] = v[0]['label']
                else:
                    flat[k] = v
        return flat

In [29]:
def format_json_string(json_string):
    json_string = json_string.replace('\n', '<br>')
    result = []
    for index, char in enumerate(json_string):
        if char == " " and (index == 0 or json_string[index - 1] != ":"):
            result.append("&nbsp;&nbsp;")
        else:
            result.append(char)

    json_string = "".join(result)
    return json_string